### 당신은 해외로 사업을 확장하고자 하는 기업에서 Data Engineer로 일하고 있습니다. 경영진에서 GDP가 높은 국가들을 대상으로 사업성을 평가하려고 합니다.

* 이 자료는 앞으로 경영진에서 지속적으로 요구할 것으로 생각되기 때문에 자동화된 스크립트를 만들어야 합니다.

# 추출 -> 변환 -> 로드

In [50]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
import sqlite3  # sqlite3 모듈 임포트# 모듈 임포트
import re

# 로그 기록 함수
def log_progress(message, etl_path):
    timestamp_format = '%Y-%b-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # 현재 시간 가져오기 
    timestamp = now.strftime(timestamp_format) 
    with open(os.path.join(etl_path, 'etl_project_log.txt'), "a") as f: 
        f.write(f"{timestamp}, {message}\n")

### 추출

In [51]:

# 추출 (Extract) 함수
def extract(url, table_attribs, etl_path):
    log_progress('데이터 추출 시작', etl_path)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
    if table is None:
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")


    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[3:]:  
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=table_attribs)
    
    #country 별 region dataset
    regions=pd.read_csv('/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/region.csv')

    log_progress('데이터 추출 완료', etl_path)
    return df

### 변환

In [52]:
# 변환 (Transform) 함수
def transform(df, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    # GDP 값을 숫자로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)  # 단위를 억 달러로 변환
    df.drop(columns=['GDP_USD_millions'], inplace=True)  # 원래 컬럼 삭제
    df = pd.merge(df, regions, on='Country', how='left')
    log_progress('데이터 변환 완료', etl_path)
    return df


### 로드

In [53]:

# 로드 (Load) 함수 - CSV 파일로 저장
def load_to_csv(df, csv_path, etl_path): 
    log_progress('CSV 파일로 데이터 저장 시작', etl_path)
    df.to_csv(csv_path, index=False)
    log_progress('CSV 파일로 데이터 저장 완료', etl_path)

# 로드 (Load) 함수 - 데이터베이스에 저장 (sqlite3 이용)
def load_to_db(df, sql_connection, table_name, etl_path):
    log_progress('데이터베이스로 데이터 저장 시작', etl_path)
    cursor = sql_connection.cursor()

    # 테이블 생성
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Country TEXT,
            GDP_USD_billions REAL
        )
    ''')

    # 데이터 삽입
    for row in df.itertuples(index=False):
        cursor.execute(f'''
            INSERT INTO {table_name} (Country, GDP_USD_billions)
            VALUES (?, ?)
        ''', (row.Country, row.GDP_USD_billions))
    
    sql_connection.commit()
    log_progress('데이터베이스로 데이터 저장 완료', etl_path)

# JSON 파일로 로드하는 함수
def load_gdp_data(df, etl_path):
    log_progress('JSON 파일로 데이터 저장 시작', etl_path)
    df.to_json(os.path.join(etl_path, 'Countries_by_GDP.json'), orient='records', lines=True)
    log_progress('JSON 파일로 데이터 저장 완료', etl_path)


In [54]:

# 데이터베이스 쿼리 실행 함수
def run_query(query_statement, sql_connection, etl_path):
    log_progress(f'쿼리 실행: {query_statement}', etl_path)
    query_output = pd.read_sql(query_statement, sql_connection)  # 쿼리 실행 결과를 데이터 프레임으로 변환
    print(query_output)
    log_progress('쿼리 실행 완료', etl_path)

# 메인 함수
def main():
    # 지정된 경로에 ETL 폴더 생성
    etl_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL'

    if not os.path.exists(etl_path):
        os.makedirs(etl_path)

    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    table_attribs = ["Country", "GDP_USD_millions"]
    db_name = os.path.join(etl_path, 'World_Economies.db')
    table_name = 'Countries_by_GDP'
    csv_path = os.path.join(etl_path, 'Countries_by_GDP.csv')

    log_progress('ETL 프로세스 시작', etl_path)

    # 추출 단계
    df = extract(url, table_attribs, etl_path)
    load_gdp_data(df, etl_path)

    # 변환 단계
    df = transform(df, etl_path)

    # 로드 단계
    load_to_csv(df, csv_path, etl_path)
    sql_connection = sqlite3.connect(db_name)
    load_to_db(df, sql_connection, table_name, etl_path)

    # 쿼리 실행
    query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
    run_query(query_statement, sql_connection, etl_path)

    sql_connection.close()
    log_progress('ETL 프로세스 완료', etl_path)
    log_progress('------------------------------------------', etl_path)

if __name__ == "__main__":
    main()


          Country  GDP_USD_billions
0   United States          28781.08
1           China          18532.63
2         Germany           4591.10
3           Japan           4110.45
4           India           3937.01
..            ...               ...
65      Guatemala            110.04
66           Oman            108.93
67       Bulgaria            107.93
68          Kenya            104.00
69      Venezuela            102.33

[70 rows x 2 columns]


In [ ]:
dir='/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/merged.csv'
merged=pd.read_csv(dir)
top_5_mean = merged.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')
top_5_mean_sorted = top_5_mean.sort_values(by='Top5_Avg_GDP_USD_billions', ascending=False)

top_5_mean_sorted

* EMEA (Europe, the Middle East and Africa)
* NA (North America)
* LATAM (Latin America)
* APAC (Asia-Pacific)

1. IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)



2. 국가별 GDP를 확인할 수 있는 테이블을 만드세요.

3. 해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.

4. GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.



5. IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.

6. GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.



7. 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다. 

In [64]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
import sqlite3  # sqlite3 모듈 임포트# 모듈 임포트
import re

# 로그 기록 함수
def log_progress(message, etl_path):
    timestamp_format = '%Y-%b-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # 현재 시간 가져오기 
    timestamp = now.strftime(timestamp_format) 
    with open(os.path.join(etl_path, 'etl_project_log.txt'), "a") as f: 
        f.write(f"{timestamp}, {message}\n")
        
def transform(df, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    # GDP 값을 숫자로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)  # 단위를 억 달러로 변환
    df.drop(columns=['GDP_USD_millions'], inplace=True)  # 원래 컬럼 삭제
    log_progress('데이터 변환 완료', etl_path)
    return df

# 추출 (Extract) 함수
def extract(url, table_attribs, etl_path):
    log_progress('데이터 추출 시작', etl_path)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
    if table is None:
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")


    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[3:]:  
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=table_attribs)
    
    log_progress('데이터 추출 완료', etl_path)
    return df
# 변환 (Transform) 함수
def transform(df, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    # GDP 값을 숫자로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)  # 단위를 억 달러로 변환
    df.drop(columns=['GDP_USD_millions'], inplace=True)  # 원래 컬럼 삭제
    log_progress('데이터 변환 완료', etl_path)
    return df

# 로드 (Load) 함수 - CSV 파일로 저장
def load_to_csv(df, csv_path, etl_path): 
    log_progress('CSV 파일로 데이터 저장 시작', etl_path)
    df.to_csv(csv_path, index=False)
    log_progress('CSV 파일로 데이터 저장 완료', etl_path)

# 로드 (Load) 함수 - 데이터베이스에 저장 (sqlite3 이용)
def load_to_db(df, sql_connection, table_name, etl_path):
    log_progress('데이터베이스로 데이터 저장 시작', etl_path)
    cursor = sql_connection.cursor()

    # 테이블 생성
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Country TEXT,
            GDP_USD_billions REAL
        )
    ''')

    # 데이터 삽입
    for row in df.itertuples(index=False):
        cursor.execute(f'''
            INSERT INTO {table_name} (Country, GDP_USD_billions)
            VALUES (?, ?)
        ''', (row.Country, row.GDP_USD_billions))
    
    sql_connection.commit()
    log_progress('데이터베이스로 데이터 저장 완료', etl_path)

# JSON 파일로 로드하는 함수
def load_gdp_data(df, etl_path):
    log_progress('JSON 파일로 데이터 저장 시작', etl_path)
    df.to_json(os.path.join(etl_path, 'Countries_by_GDP.json'), orient='records', lines=True)
    log_progress('JSON 파일로 데이터 저장 완료', etl_path)


# 데이터베이스 쿼리 실행 함수
def run_query(query_statement, sql_connection, etl_path):
    log_progress(f'쿼리 실행: {query_statement}', etl_path)
    query_output = pd.read_sql(query_statement, sql_connection)  # 쿼리 실행 결과를 데이터 프레임으로 변환
    print(query_output)
    log_progress('쿼리 실행 완료', etl_path)

# 메인 함수
def main():
    # 지정된 경로에 ETL 폴더 생성
    etl_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL'

    if not os.path.exists(etl_path):
        os.makedirs(etl_path)

    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    table_attribs = ["Country", "GDP_USD_millions"]
    db_name = os.path.join(etl_path, 'World_Economies.db')
    table_name = 'Countries_by_GDP'
    csv_path = os.path.join(etl_path, 'Countries_by_GDP.csv')

    log_progress('ETL 프로세스 시작', etl_path)

    # 추출 단계
    df = extract(url, table_attribs, etl_path)
    load_gdp_data(df, etl_path)

    # 변환 단계
    df = transform(df, etl_path)

    # 로드 단계
    load_to_csv(df, csv_path, etl_path)
    sql_connection = sqlite3.connect(db_name)
    load_to_db(df, sql_connection, table_name, etl_path)

    # 쿼리 실행
    query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
    run_query(query_statement, sql_connection, etl_path)

    sql_connection.close()
    log_progress('ETL 프로세스 완료', etl_path)
    log_progress('------------------------------------------', etl_path)

if __name__ == "__main__":
    main()



           Country  GDP_USD_billions
0    United States          28781.08
1            China          18532.63
2          Germany           4591.10
3            Japan           4110.45
4            India           3937.01
..             ...               ...
275      Guatemala            110.04
276           Oman            108.93
277       Bulgaria            107.93
278          Kenya            104.00
279      Venezuela            102.33

[280 rows x 2 columns]


In [70]:

import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime

# 로그 기록 함수
def log_progress(message, etl_path):
    # 시간 형식 정의
    timestamp_format = '%Y-%b-%d-%H:%M:%S'  # 년-월이름-일-시간-분-초 형식
    now = datetime.now()  # 현재 시간 가져오기
    timestamp = now.strftime(timestamp_format)  # 형식에 맞게 시간 문자열로 변환
    # 로그 파일에 기록
    with open(os.path.join(etl_path, 'etl_project_log.txt'), "a") as f:
        f.write(f"{timestamp}, {message}\n")

# 추출 (Extract) 함수
def extract(url, table_attribs, etl_path):
    log_progress('데이터 추출 시작', etl_path)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

  # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
    if table is None:
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")


    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[3:]:  
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=table_attribs)
    log_progress('데이터 추출 완료', etl_path)
    return df

# 변환 (Transform) 함수
def transform(df, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    # GDP 값을 숫자로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)  # 단위를 억 달러로 변환
    df.drop(columns=['GDP_USD_millions'], inplace=True)  # 원래 컬럼 삭제
    log_progress('데이터 변환 완료', etl_path)
    return df


# 로드 (Load) 함수 - CSV 파일로 저장
def load_to_csv(df, csv_path, etl_path):
    log_progress('CSV 파일로 데이터 저장 시작', etl_path)
    df.to_csv(csv_path, index=False)  # CSV 파일로 저장
    log_progress('CSV 파일로 데이터 저장 완료', etl_path)

# JSON 파일로 로드하는 함수
def load_gdp_data(df, etl_path):
    log_progress('JSON 파일로 데이터 저장 시작', etl_path)
    # JSON 파일로 저장
    df.to_json(os.path.join(etl_path, 'Countries_by_GDP.json'), orient='records', lines=True)
    log_progress('JSON 파일로 데이터 저장 완료', etl_path)

# 메인 함수
def main():
    # 지정된 경로에 ETL 폴더 생성
    etl_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL'

    if not os.path.exists(etl_path):
        os.makedirs(etl_path)

    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    table_attribs = ["Country", "GDP_USD_millions"]
    csv_path = os.path.join(etl_path, 'Countries_by_GDP.csv')

    log_progress('ETL 프로세스 시작', etl_path)

    # 추출 단계
    df = extract(url, table_attribs, etl_path)
    load_gdp_data(df, etl_path)

    # 변환 단계
    df = transform(df, etl_path)

    # 로드 단계
    load_to_csv(df, csv_path, etl_path)

    log_progress('ETL 프로세스 완료', etl_path)
    log_progress('------------------------------------------', etl_path)

if __name__ == "__main__":
    main()


# 지역별 추출
북아메리카.
남아메리카
아프리카
유럽
아시아
오세아니아

In [184]:
url='/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL/Countries_by_GDP.csv'
df=pd.read_csv(url)
df

,Country,GDP_USD_billions
0,United States,28781.08
1,China,18532.63
2,Germany,4591.10
3,Japan,4110.45
4,India,3937.01
...,...,...
189,Kiribati,0.31
190,Palau,0.31
191,Marshall Islands,0.30
192,Nauru,0.16


In [185]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_African_countries_by_GDP_(nominal)"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    afreca = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return afreca


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, afreca):

    gdp_data = afreca.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
afreca = make_df()
gdp_data = transform_data(preprocess_gdp, afreca)


afreca=afreca[['Country']]
afreca['region']='afreca'
afreca.head(3)

,Country,region
0,South Africa,afreca
1,Egypt,afreca
2,Algeria,afreca


### Asian

In [186]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_Asian_countries_by_GDP"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    asian = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return asian


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, asian):

    gdp_data = asian.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
asian = make_df()
gdp_data = transform_data(preprocess_gdp, asian)


asian=asian[['Country']]
asian['region']='asia'
asian.head(3)

ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

### European

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_European_countries_by_GDP_growth"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    European = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return European


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, European):

    gdp_data = European.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
European = make_df()
gdp_data = transform_data(preprocess_gdp, European)


European=European[['Country']]
European['region']='europ'
European.head(3)

,Country,region
0,Faroe Islands,europ
1,Ireland,europ
2,Monaco,europ


### LATAM (라틴아메리카)


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_South_American_countries_and_dependencies_by_GDP_(PPP)"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    SA = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return SA


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, SA):

    gdp_data = SA.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
SA = make_df()
gdp_data = transform_data(preprocess_gdp, SA)


SA['Country']=SA[['Country']]
SA=SA[['Country']]
SA['region']='SA'
SA.head(3)

,Country,region
0,South America,SA
1,Brazil,SA
2,Argentina,SA


### North American

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_North_American_countries_by_GDP_(PPP)"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    NA = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return NA


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, NA):

    gdp_data = NA.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
NA = make_df()
gdp_data = transform_data(preprocess_gdp, NA)


NA=NA[['Country']]
NA['region']='NA'
NA.head(3)

,Country,region
0,North America,NA
1,United States,NA
2,Mexico,NA


EMEA (유럽, 중동, 아프리카)
NA (북미)
LATAM (라틴아메리카)
APAC(아시아 태평양)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from etl_project_gdp import *


def make_df():
    url = "https://en.wikipedia.org/wiki/List_of_Oceanian_countries_by_GDP_growth"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # HTML에서 테이블을 DataFrame으로 읽어오기
    table = soup.find("table", class_="wikitable")
    Oceania = pd.read_html(StringIO(str(table)))[0]  # 첫 번째 테이블만 가져오기


    return Oceania


# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}

    # 국가명 처리
    if pd.notna(row["Country"]):
        if row["Country"] != "World":
            data["Country"] = row["Country"]

    if len(data) > 1:
        return data


def transform_data(preprocess_gdp, Oceania):

    gdp_data = Oceania.apply(preprocess_gdp, axis=1).tolist()
    gdp_data = [entry for entry in gdp_data if entry is not None]

    return gdp_data


# 크롤링 후, 데이터 전처리 적용
Oceania = make_df()
gdp_data = transform_data(preprocess_gdp, Oceania)


Oceania['Country']=Oceania[['Country']]
Oceania=Oceania[['Country']]
Oceania['region']='Oceania'
Oceania.head(3)

,Country,region
0,Vanuatu,Oceania
1,Nauru,Oceania
2,Solomon Islands,Oceania


In [ ]:
regions = pd.concat([European,afreca,asian, Oceania,NA,SA], ignore_index=True)
regions.to_csv('region.csv')

In [ ]:
df

,Country,GDP_USD_billions
0,United States,28781.08
1,China,18532.63
2,Germany,4591.10
3,Japan,4110.45
4,India,3937.01
...,...,...
189,Kiribati,0.31
190,Palau,0.31
191,Marshall Islands,0.30
192,Nauru,0.16


In [189]:
# df와 regions를 Country를 기준으로 병합
regions=pd.read_csv('/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/region.csv')
merged_df = pd.merge(df, regions, on='Country', how='left')

# Region 컬럼명 변경
#merged_df.rename(columns={'region': 'Region'}, inplace=True)


,Country,Region
0,United States,North America
1,ChiNorth America,Asia
2,Germany,Europe
3,Japan,Asia
4,India,Asia
...,...,...
189,Kiribati,Oceania
190,Palau,Oceania
191,Marshall Islands,Oceania
192,North Americauru,Oceania


In [ ]:
merged_df

,Country,GDP_USD_billions,Region
0,United States,28781.08,NA
1,China,18532.63,asia
2,Germany,4591.10,europ
3,Japan,4110.45,asia
4,India,3937.01,asia
...,...,...,...
189,Kiribati,0.31,Oceania
190,Palau,0.31,NaN
191,Marshall Islands,0.30,Oceania
192,Nauru,0.16,Oceania


,Unnamed: 0,Country,GDP_USD_billions,Region
0,0,United States,28781.08,North America
1,1,ChiNorth America,18532.63,Asia
2,2,Germany,4591.10,Europe
3,3,Japan,4110.45,Asia
4,4,India,3937.01,Asia
...,...,...,...,...
189,189,Kiribati,0.31,Oceania
190,190,Palau,0.31,Oceania
191,191,Marshall Islands,0.30,Oceania
192,192,North Americauru,0.16,Oceania


In [ ]:
url='/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/merged.csv'
merged=pd.read_csv(url)
#merged=merged[['Country','GDP_USD_billions','Region']]
merged
top_5_mean = merged.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')
top_5_mean_sorted = top_5_mean.sort_values(by='Top5_Avg_GDP_USD_billions', ascending=False)

top_5_mean_sorted

/var/folders/gv/h6tkbjcs4j325ypvzj8m08qc0000gn/T/ipykernel_38706/1220636639.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_5_mean = merged.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')


,Region,Top5_Avg_GDP_USD_billions
3,North America,6655.536
1,Asia,5963.346
2,Europe,3120.248
5,South America,787.590
4,Oceania,417.440
0,Afreeca,289.094


In [208]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
import sqlite3  # sqlite3 모듈 임포트# 모듈 임포트
import re

# 로그 기록 함수
def log_progress(message, etl_path):
    timestamp_format = '%Y-%b-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # 현재 시간 가져오기 
    timestamp = now.strftime(timestamp_format) 
    with open(os.path.join(etl_path, 'etl_project_log.txt'), "a") as f: 
        f.write(f"{timestamp}, {message}\n")
        

# 추출 (Extract) 함수
def extract(url, table_attribs, etl_path):
    log_progress('데이터 추출 시작', etl_path)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
    if table is None:
        raise ValueError("Unable to find the sortable table on the Wikipedia page.")


    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[3:]:  
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=table_attribs)
    
    log_progress('데이터 추출 완료', etl_path)
    return df


# 변환 (Transform) 함수
def transform(df, regions_path, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    # GDP 값을 숫자로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)  # 단위를 억 달러로 변환
    df.drop(columns=['GDP_USD_millions'], inplace=True)  # 원래 컬럼 삭제
    #df에 region 부여
    df = pd.merge(df, regions, on='Country', how='left') 
    top_5_mean = df.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')
    top_5_mean_sorted = top_5_mean.sort_values(by='Top5_Avg_GDP_USD_billions', ascending=False)
    print('-' * 36)
    print("각 Region별로 top5 국가의 GDP 평균")
    print(top_5_mean_sorted)
    log_progress('데이터 변환 완료', etl_path)
    print(df)
    return df



# 로드 (Load) 함수 - CSV 파일로 저장
def load_to_csv(df, csv_path, etl_path): 
    log_progress('CSV 파일로 데이터 저장 시작', etl_path)
    df.to_csv(csv_path, index=False)
    log_progress('CSV 파일로 데이터 저장 완료', etl_path)

# 로드 (Load) 함수 - 데이터베이스에 저장 (sqlite3 이용)
def load_to_db(df, sql_connection, table_name, etl_path):
    log_progress('데이터베이스로 데이터 저장 시작', etl_path)
    cursor = sql_connection.cursor()

    # 테이블 생성
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Country TEXT,
            GDP_USD_billions REAL,
            Region TEXT
        )
    ''')

    # 데이터 삽입
    for row in df.itertuples(index=False):
        cursor.execute(f'''
            INSERT INTO {table_name} (Country, GDP_USD_billions,Region)
            VALUES (?, ?, ?)
        ''', (row.Country, row.GDP_USD_billions,row.Region))
    
    sql_connection.commit()
    log_progress('데이터베이스로 데이터 저장 완료', etl_path)

# JSON 파일로 로드하는 함수
def load_gdp_data(df, etl_path):
    log_progress('JSON 파일로 데이터 저장 시작', etl_path)
    df.to_json(os.path.join(etl_path, 'Countries_by_GDP.json'), orient='records', lines=True)
    log_progress('JSON 파일로 데이터 저장 완료', etl_path)


# 데이터베이스 쿼리 실행 함수
def run_query(query_statement, sql_connection, etl_path):
    log_progress(f'쿼리 실행: {query_statement}', etl_path)
    query_output = pd.read_sql(query_statement, sql_connection)  # 쿼리 실행 결과를 데이터 프레임으로 변환
    print(query_output)
    log_progress('쿼리 실행 완료', etl_path)

# 메인 함수
def main():
    # 지정된 경로에 ETL 폴더 생성
    etl_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL'

    if not os.path.exists(etl_path):
        os.makedirs(etl_path)

    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    regions_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/region.csv'
    table_attribs = ["Country", "GDP_USD_millions"]
    db_name = os.path.join(etl_path, 'World_Economies.db')
    table_name = 'Countries_by_GDP'
    csv_path = os.path.join(etl_path, 'Countries_by_GDP.csv')

    log_progress('ETL 프로세스 시작', etl_path)

    # 추출 단계
    df = extract(url, table_attribs, etl_path)
    load_gdp_data(df, etl_path)

    # 변환 단계
    df = transform(df, regions_path,etl_path)

    # 로드 단계
    load_to_csv(df, csv_path, etl_path)
    sql_connection = sqlite3.connect(db_name)
    #load_to_db(df, sql_connection, table_name, etl_path)

    # 쿼리 실행
    print('-' * 36)
    print("GDP가 100B USD 이상인 국가들")
    log_progress("GDP가 100B USD 이상인 국가 출력",etl_path)
    query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
    run_query(query_statement, sql_connection, etl_path)

    sql_connection.close()
    log_progress('ETL 프로세스 완료', etl_path)
    log_progress('-' * 36, etl_path)

if __name__ == "__main__":
    main()



------------------------------------
각 Region별로 top5 국가의 GDP 평균
          Region  Top5_Avg_GDP_USD_billions
3  North America                   6226.312
2         Europe                   3120.248
1           Asia                   2417.412
5  South America                    691.056
4        Oceania                    417.440
0        Afreeca                    289.094
              Country  GDP_USD_billions         Region
0       United States          28781.08  North America
1               China          18532.63            NaN
2             Germany           4591.10         Europe
3               Japan           4110.45           Asia
4               India           3937.01           Asia
..                ...               ...            ...
189          Kiribati              0.31        Oceania
190             Palau              0.31        Oceania
191  Marshall Islands              0.30        Oceania
192             Nauru              0.16            NaN
193            Tuvalu  

/var/folders/gv/h6tkbjcs4j325ypvzj8m08qc0000gn/T/ipykernel_38706/136096712.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_5_mean = df.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')


In [206]:
df

,Country,GDP_USD_billions
0,United States,28781.08
1,China,18532.63
2,Germany,4591.10
3,Japan,4110.45
4,India,3937.01
...,...,...
189,Kiribati,0.31
190,Palau,0.31
191,Marshall Islands,0.30
192,Nauru,0.16


In [271]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import os
import sqlite3
from datetime import datetime
from IPython.display import display

# 로그 기록 함수
def log_progress(message, etl_path):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(os.path.join(etl_path, 'etl_project_log.txt'), "a") as f:
        f.write(f"{timestamp}, {message}\n")

# 데이터 추출 함수
def extract(url, table_attribs, etl_path):
    log_progress('데이터 추출 시작', etl_path)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    # 적절한 테이블 선택
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
    if table is None:
        log_progress("Wikipedia 페이지에서 테이블을 찾을 수 없음", etl_path)
        raise ValueError("Wikipedia 페이지에서 테이블을 찾을 수 없음")

    # 테이블 모든 행 추출 
    rows = table.find_all('tr')
    data = []

    # 각 행의 데이터를 파싱
    for row in rows[3:]:   
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
            
        country = cols[0].text.strip()
        gdp_text = cols[1].text.strip()
        gdp = re.sub(r'\[.*?\]', '', gdp_text).replace(',', '')
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue 

        data.append([country, gdp])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=table_attribs)
    log_progress('데이터 추출 완료', etl_path)
    return df

# 데이터 변환 함수
def transform(df, regions_path, etl_path):
    log_progress('데이터 변환 시작', etl_path)
    
    # GDP 값을 숫자로 변환 및 단위를 억 달러로 변환
    df["GDP_USD_millions"] = df["GDP_USD_millions"].astype(float)
    df["GDP_USD_billions"] = df["GDP_USD_millions"].div(1000).round(2)
    df.drop(columns=['GDP_USD_millions'], inplace=True)

    # Region 데이터 병합
    regions = pd.read_csv(regions_path)
    df = pd.merge(df, regions, on='Country', how='left')

    # 각 Region별로 top5 국가의 GDP 평균 계산
    top_5_mean = df.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')
    top_5_mean_sorted = top_5_mean.sort_values(by='Top5_Avg_GDP_USD_billions', ascending=False)

    print('-' * 36)
    print("각 Region별로 top5 국가의 GDP 평균")
    display(top_5_mean_sorted)

    log_progress('데이터 변환 완료', etl_path)
    return df

# 데이터 로드 함수 (CSV)
def load_to_csv(df, csv_path, etl_path):
    log_progress('CSV 파일로 데이터 저장 시작', etl_path)
    df.to_csv(csv_path, index=False)
    log_progress('CSV 파일로 데이터 저장 완료', etl_path)

# 데이터 로드 함수 (JSON)
def load_gdp_data_to_json(df, filename, etl_path):
    log_progress('데이터를 JSON 파일로 저장 시작', etl_path)
    df.to_json(filename, orient='records', lines=True, force_ascii=False)
    log_progress('데이터를 JSON 파일로 저장 완료', etl_path)

# 쿼리 실행 함수
def run_query(query_statement, db_name, etl_path):
    log_progress(f"쿼리 실행: {query_statement}", etl_path)
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query_statement, conn)
    conn.close()
    display(df)

# 테이블 생성 및 데이터 삽입 함수
def create_and_insert_table(df, db_name, table_name, etl_path):
    log_progress(f"{table_name} 테이블 생성 및 데이터 삽입 시작", etl_path)
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    log_progress(f"{table_name} 테이블 생성 및 데이터 삽입 완료", etl_path)

# ETL 프로세스 실행 함수
def etl_process():
    etl_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/ETL'
    if not os.path.exists(etl_path):
        os.makedirs(etl_path)
    db_name = os.path.join(etl_path, 'World_Economies.db')
    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    table_attribs = ["Country", "GDP_USD_millions"]
    table_name = 'Countries_by_GDP'
    regions_path = '/Users/admin/Documents/GitHub/softeer_wiki/missions/w1/region.csv'
    csv_path = os.path.join(etl_path, 'Countries_by_GDP.csv')
    json_path = os.path.join(etl_path, 'Countries_by_GDP.json')
    log_progress('ETL 프로세스 시작', etl_path)

    # 데이터 추출
    df = extract(url, table_attribs, etl_path)
    load_gdp_data_to_json(df, json_path, etl_path)

    # 데이터 변환
    df = transform(df, regions_path, etl_path)
    
    # 테이블 생성 및 데이터 삽입
    create_and_insert_table(df, db_name, table_name, etl_path)
    
    print('-' * 36)
    print("GDP가 100B USD 이상인 국가들")
    log_progress("GDP가 100B USD 이상인 국가 출력", etl_path)
    query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
    run_query(query_statement, db_name, etl_path)

    # 데이터 로드
    load_to_csv(df, csv_path, etl_path)

    log_progress('ETL 프로세스 완료', etl_path)
    log_progress('------------------------------------------', etl_path)

if __name__ == "__main__":
    etl_process()


------------------------------------
각 Region별로 top5 국가의 GDP 평균


/var/folders/gv/h6tkbjcs4j325ypvzj8m08qc0000gn/T/ipykernel_38706/67862508.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_5_mean = df.groupby('Region').apply(lambda x: x.nlargest(5, 'GDP_USD_billions')['GDP_USD_billions'].mean()).reset_index(name='Top5_Avg_GDP_USD_billions')


,Region,Top5_Avg_GDP_USD_billions
3,North America,6226.312
1,Asia,5963.346
2,Europe,3120.248
5,South America,691.056
4,Oceania,417.440
0,Afreeca,289.094


------------------------------------
GDP가 100B USD 이상인 국가들


,Country,GDP_USD_billions,Region
0,United States,28781.08,North America
1,China,18532.63,Asia
2,Germany,4591.10,Europe
3,Japan,4110.45,Asia
4,India,3937.01,Asia
...,...,...,...
65,Guatemala,110.04,North America
66,Oman,108.93,Asia
67,Bulgaria,107.93,Europe
68,Kenya,104.00,Afreeca
